In [0]:
!nvidia-smi

In [0]:
!add-apt-repository ppa:alessandro-strada/ppa

In [0]:
!python main.py videoed extract-video --input-file workspace/data_dst.* --output-dir workspace/data_dst/ --fps 0 --output-ext jpg

In [0]:
!python main.py videoed extract-video --input-file workspace/data_src.* --output-dir workspace/data_src/ --fps 0 --output-ext jpg

# Denoise Frames

In [0]:
!python main.py videoed denoise-image-sequence --input-dir workspace/data_dst --factor 7

In [0]:
!python main.py videoed denoise-image-sequence --input-dir workspace/data_src --factor 7

# Detect Faces and Align

In [0]:
#@markdown Select the Type of **Detector** (s3fd recommended) and **FaceType**
Detector = "s3fd" #@param ["s3fd", "manual"]
FaceType = "whole_face" #@param ["head", "half_face", "full_face", "whole_face", "mark_only"]

In [0]:
!python main.py extract --input-dir workspace/data_dst --output-dir workspace/data_dst/aligned --face-type whole_face --detector s3fd --force-gpu-idxs 0 --max-faces-from-image 0 --image-size 512 --jpeg-quality 90 --no-output-debug

In [0]:
!python main.py extract --input-dir workspace/data_src --output-dir workspace/data_src/aligned --face-type whole_face --detector s3fd --force-gpu-idxs 0 --max-faces-from-image 0 --image-size 512 --jpeg-quality 90 --no-output-debug

# Sort Aligned Faces

In [0]:
#@markdown Select **sort type** (absdiff recommended)
sort_type = "absdiff" #@param ["absdiff", "brightness", "hist", "hist-dissim", "face-yaw", "face-pitch", "vggface", "blur", "final", "final-no-blur"]

In [0]:
!python main.py sort --input-dir workspace/data_dst/aligned --by absdiff

In [0]:
!python main.py sort --input-dir workspace/data_src/aligned --by absdiff

#  Faceset Enhance

In [0]:
!python main.py facesettool enhance --input-dir /workspace/data_dst/aligned

In [0]:
!python main.py facesettool enhance --input-dir /workspace/data_src/aligned

# Train Model

In [0]:
Model = "SAEHD" #@param ["SAEHD", "SAE", "Quick96", "H128", "LIAEF128", "DF"]

In [0]:
!python main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model SAEHD

# Convert Frames

In [0]:
!python main.py merge --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --output-mask-dir workspace/data_dst/merged_mask --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model SAEHD

# Get Final Result Video

In [0]:
!python main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4 --bitrate 16